Sacar mi historial de partidas SUMMONER-V4, MATCH-V5
o 
info de los jugadores de una división concreta LEAGUE-V4 division

In [1]:
import requests
import pandas as pd
from creds import api_key
import re

In [2]:
url_summoner = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Masgt7'

Nos centramos en SUMMONER-V4que tiene que ver con cualquier cosa que haga una cuenta en concreto

In [3]:
url_summoner = url_summoner + '?api_key=' + api_key

In [4]:
r = requests.get (url_summoner)
r.status_code

200

In [5]:
player_info = r.json()
print (player_info)

{'id': '7xKp5xDyqk1u0_RKrh8SdaSHYhjpThiq7XXJscT2DQx273I', 'accountId': 'NcGfPSobpg1gNOsZBjrQNCVyeIg7MyijVoglw0WBQioOST0', 'puuid': 'uFIekhnHnZjaIXG_IG5y2Kb_UbBsEv57LksBUe6qMh0h0MPBKg8xoWtsbQkvXcPhlHLNGn8xZwqzGQ', 'name': 'Masgt7', 'profileIconId': 940, 'revisionDate': 1700939375072, 'summonerLevel': 74}


Obtenida la info de mi cuenta, ahora hacemos otra request para obtener identificadores de mis últimas partidas a partir del 'puuid'

In [6]:
puuid = player_info['puuid']

In [7]:
url_match_history = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?type=normal&start=0&count=20&api_key=' + api_key #Limitamos la lista de partidas a 20
r = requests.get(url_match_history)

In [8]:
if r.status_code == 200:
    game_id = r.json()
    print (game_id)
else:
    print ('Error', r.status_code, ':',)  

['EUW1_6691848158', 'EUW1_6688283685', 'EUW1_6688256964', 'EUW1_6688236897', 'EUW1_6688218767', 'EUW1_6686773929', 'EUW1_6686744190', 'EUW1_6684157765', 'EUW1_6684133215', 'EUW1_6682813114', 'EUW1_6682767095', 'EUW1_6682519608', 'EUW1_6678781224', 'EUW1_6678748896', 'EUW1_6678696426', 'EUW1_6678677984', 'EUW1_6678647009', 'EUW1_6677476957', 'EUW1_6677447766', 'EUW1_6673723132']


Vamos a sacar ahora información de cada partida y almacenarla en ficheros .csv por separado

In [9]:
counter = 1
for i in game_id:
    url_game_info = 'https://europe.api.riotgames.com/lol/match/v5/matches/' + i + '?api_key=' + api_key
    game_info = requests.get(url_game_info)
    index_player = game_info.json()['metadata']['participants'].index(puuid)
    game_data = game_info.json()['info']['participants'][index_player]
    game_df = pd.json_normalize(game_data)
    game_df.to_csv ('games\\game' + str(counter) + '.csv', sep = ',', mode = 'x')
    counter += 1

Cargamos una lista con DF a partir de los CSV

In [10]:
list_df = []

for i in range(len(game_id)):
    list_df.append((pd.read_csv('games\\game' + str (i+1) + '.csv')))
    print (list_df[i].shape)

(1, 265)
(1, 262)
(1, 264)
(1, 264)
(1, 264)
(1, 264)
(1, 263)
(1, 268)
(1, 269)
(1, 268)
(1, 269)
(1, 269)
(1, 268)
(1, 268)
(1, 267)
(1, 267)
(1, 267)
(1, 267)
(1, 266)
(1, 268)


No tienen las mismas columnas, de modo que vamos a quedarnos únicamente con las que tengan en común

In [11]:
common_columns = list(set.intersection(*(set(df.columns) for df in list_df)))
common_columns[0:10] #Vemos algunas

['challenges.flawlessAces',
 'challenges.perfectDragonSoulsTaken',
 'spell3Casts',
 'assists',
 'item0',
 'challenges.quickCleanse',
 'challenges.dancedWithRiftHerald',
 'challenges.skillshotsDodged',
 'trueDamageDealtToChampions',
 'challenges.blastConeOppositeOpponentCount']

In [12]:
df_joined = pd.concat(list_df)
df_joined = df_joined [common_columns]

df_joined.head()

,challenges.flawlessAces,challenges.perfectDragonSoulsTaken,spell3Casts,assists,item0,challenges.quickCleanse,challenges.dancedWithRiftHerald,challenges.skillshotsDodged,trueDamageDealtToChampions,challenges.blastConeOppositeOpponentCount,...,visionWardsBoughtInGame,wardsKilled,challenges.mejaisFullStackInTime,challenges.wardsGuarded,challenges.riftHeraldTakedowns,challenges.epicMonsterKillsWithin30SecondsOfSpawn,playerScore10,playerAugment2,missions.playerScore3,challenges.getTakedownsInAllLanesEarlyJungleAsLaner
0,0,0,24,16,6672,0,0,40,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,42,28,3084,0,0,53,3204,0,...,0,0,0,0,0,0,0,0,0,0
0,1,0,32,24,6672,0,0,93,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,38,9,6632,0,0,69,1800,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,44,14,6632,0,0,72,2753,0,...,0,0,0,0,0,0,0,0,0,0


Eliminamos las palabras previas al punto

In [13]:
def remove_dot (string):
    pattern = r'.*?\.(.*)'
    result = re.match(pattern, string)
    if result:
        return result.group(1)
    else:
        return string

In [14]:
df_columns = list(df_joined.columns)

for i,j in enumerate(df_columns):
    df_columns[i] = remove_dot(j)

df_joined.columns = df_columns

df_joined.head(1)

,flawlessAces,perfectDragonSoulsTaken,spell3Casts,assists,item0,quickCleanse,dancedWithRiftHerald,skillshotsDodged,trueDamageDealtToChampions,blastConeOppositeOpponentCount,...,visionWardsBoughtInGame,wardsKilled,mejaisFullStackInTime,wardsGuarded,riftHeraldTakedowns,epicMonsterKillsWithin30SecondsOfSpawn,playerScore10,playerAugment2,playerScore3,getTakedownsInAllLanesEarlyJungleAsLaner
0,0,0,24,16,6672,0,0,40,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_joined.shape

(20, 260)

We have a very high dimensionality in our dataset, so now we'll check the descriptive statistics of each variable, and we'll drop the columns that we consider not relevant

In [16]:
# display(common_rows)
pd.set_option('display.max_columns', None) #Quitamos el límite de columnas para poder ver todas y evaluar cuáles son importantes
display(df_joined.describe())

,flawlessAces,perfectDragonSoulsTaken,spell3Casts,assists,item0,quickCleanse,dancedWithRiftHerald,skillshotsDodged,trueDamageDealtToChampions,blastConeOppositeOpponentCount,statPerks.defense,neutralMinionsKilled,playerScore3,killsUnderOwnTurret,damageDealtToBuildings,completeSupportQuestInTime,multiTurretRiftHeraldCount,inhibitorKills,totalAllyJungleMinionsKilled,enemyVisionPings,kills,trueDamageTaken,alliedJungleMonsterKills,dodgeSkillShotsSmallWindow,immobilizeAndKillWithAlly,totalHealsOnTeammates,takedowns,teamPosition,wardTakedowns,turretKills,visionScorePerMinute,takedownsFirstXMinutes,playerScore4,playerScore11,12AssistStreakCount,playerScore9,longestTimeSpentLiving,saveAllyFromDeath,acesBefore15Minutes,pushPings,playerAugment3,playerScore0,epicMonsterSteals,hadOpenNexus,needVisionPings,summoner1Casts,elderDragonMultikills,quickFirstTurret,dragonTakedowns,fullTeamTakedown,gameLength,poroExplosions,playerScore8,tripleKills,baitPings,objectivesStolen,teamElderDragonKills,knockEnemyIntoTeamAndKill,playerScore0,snowballsHit,effectiveHealAndShielding,kda,item5,teamBaronKills,playerScore6,visionScore,turretsLost,item4,playerScore7,championTransform,detectorWardsPlaced,champExperience,killAfterHiddenWithAlly,playerScore2,killsOnRecentlyHealedByAramPack,playerScore9,item6,subteamPlacement,getBackPings,totalDamageDealtToChampions,enemyJungleMonsterKills,onMyWayPings,killsOnOtherLanesEarlyJungleAsLaner,epicMonsterStolenWithoutSmite,magicDamageDealtToChampions,totalMinionsKilled,soloKills,participantId,doubleAces,enemyMissingPings,item3,killsWithHelpFromEpicMonster,epicMonsterKillsNearEnemyJungler,killedChampTookFullTeamDamageSurvived,outnumberedNexusKill,skillshotsHit,maxKillDeficit,dragonKills,playerScore7,laneMinionsFirst10Minutes,playerScore11,takedownsInAlcove,playerAugment1,champLevel,dangerPings,multikills,soloBaronKills,item1,initialBuffCount,magicDamageDealt,assistMePings,profileIcon,lostAnInhibitor,legendaryCount,kTurretsDestroyedBeforePlatesFall,totalTimeCCDealt,deaths,survivedThreeImmobilizesInFight,trueDamageDealt,multiKillOneSpell,pickKillWithAlly,turretTakedowns,twentyMinionsIn3SecondsCount,inhibitorsLost,inhibitorTakedowns,jungleCsBefore10Minutes,moreEnemyJungleThanOpponent,nexusTakedowns,killsNearEnemyTurret,doubleKills,controlWardsPlaced,takedownsBeforeJungleMinionSpawn,perfectGame,unseenRecalls,totalTimeSpentDead,quickSoloKills,spell1Casts,totalDamageShieldedOnTeammates,damageDealtToTurrets,largestCriticalStrike,firstTurretKilled,multikillsAfterAggressiveFlash,wardTakedownsBefore20M,damageTakenOnTeamPercentage,magicDamageTaken,quadraKills,placement,summoner1Id,damageDealtToObjectives,playerScore2,timeCCingOthers,totalEnemyJungleMinionsKilled,basicPings,elderDragonKillsWithOpposingSoul,summoner2Id,deathsByEnemyChamps,physicalDamageDealt,holdPings,consumablesPurchased,physicalDamageDealtToChampions,baronTakedowns,turretTakedowns,playerScore1,playerScore5,unrealKills,wardsPlaced,championId,survivedSingleDigitHpCount,takedownsInEnemyFountain,sightWardsBoughtInGame,bountyLevel,teamId,outnumberedKills,turretPlatesTaken,Unnamed: 0,totalHeal,killingSprees,goldSpent,itemsPurchased,commandPings,statPerks.flex,playerSubteamId,turretsTakenWithRiftHerald,tookLargeDamageSurvived,teamRiftHeraldKills,timePlayed,teamDamagePercentage,bountyGold,abilityUses,enemyChampionImmobilizations,totalDamageDealt,playerScore6,junglerTakedownsNearDamagedEpicMonster,buffsStolen,playerAugment4,spell2Casts,playerScore8,spell4Casts,largestMultiKill,physicalDamageTaken,playerScore10,scuttleCrabKills,visionClearedPings,damagePerMinute,baronKills,playerScore4,totalUnitsHealed,item2,nexusLost,playerScore5,takedownOnFirstTurret,allInPings,outerTurretExecutesBefore10Minutes,killParticipation,takedownsAfterGainingLevelAdvantage,largestKillingSpree,stealthWardsPlaced,goldPerMinute,statPerks.offense,objectivesStolenAssists,nexusKills,killingSprees,damageSelfMitigated,playerScore1,initialCrabCount,landSkillShotsEarlyGame,pentaKills,twoWardsOneSweeperCount,t

In [17]:
df_joined = df_joined [['kills', 'deaths', 'assists', 'kda', 'championName', 'individualPosition', 'killParticipation', 'summoner1Casts', 'summoner2Casts', 
                       'largestMultiKill', 'totalDamageDealtToChampions', 'teamDamagePercentage', 'totalDamageTaken', 'damageTakenOnTeamPercentage',
                       'largestKillingSpree', 'allInPings', 'assistMePings', 'baitPings', 
                       'basicPings', 'commandPings', 'dangerPings', 'enemyVisionPings', 'getBackPings', 'visionClearedPings', 'holdPings', 'needVisionPings', 
                       'onMyWayPings', 'gameLength', 'goldEarned', 'goldPerMinute', 'wardsPlaced', 'wardsKilled', 'win']]

In [18]:
df_joined.shape

(20, 33)

In [19]:
df_joined.reset_index(drop=True, inplace=True)
df_joined.index += 1

df_joined.head()

,kills,deaths,assists,kda,championName,individualPosition,killParticipation,summoner1Casts,summoner2Casts,largestMultiKill,totalDamageDealtToChampions,teamDamagePercentage,totalDamageTaken,damageTakenOnTeamPercentage,largestKillingSpree,allInPings,assistMePings,baitPings,basicPings,commandPings,dangerPings,enemyVisionPings,getBackPings,visionClearedPings,holdPings,needVisionPings,onMyWayPings,gameLength,goldEarned,goldPerMinute,wardsPlaced,wardsKilled,win
1,23,5,16,7.8,Tristana,Invalid,0.750000,2,3,4,23585,0.228739,12940,0.155921,10,0,0,0,0,0,0,0,0,0,0,0,0,913.553046,14571,957.009865,0,0,True
2,6,10,28,3.4,Shen,Invalid,0.772727,5,33,1,25707,0.155245,58100,0.321806,2,0,0,0,0,0,0,0,0,0,0,0,0,1419.965720,14124,596.827073,0,0,False
3,15,13,24,3.0,Xayah,Invalid,0.609375,5,6,2,27617,0.224540,30754,0.161388,3,0,0,0,0,0,0,1,0,0,0,0,0,1164.431252,15387,792.887168,0,0,False
4,19,10,9,2.8,Jax,Invalid,0.549020,6,36,2,26658,0.245217,35250,0.220436,3,0,0,0,0,2,0,0,0,0,0,0,5,1388.050799,17809,769.816702,0,0,True
5,6,10,14,2.0,Camille,Invalid,0.500000,4,16,2,13649,0.156076,24784,0.125879,2,0,0,0,0,0,0,0,0,0,0,0,0,992.416079,12063,729.311037,0,0,True


Hacemos transformaciones. Sumamos los pings en una sola columna, arreglamos los porcentajes y ponemos la duración de la partida en minutos

In [20]:
percentages = ['killParticipation','teamDamagePercentage', 'damageTakenOnTeamPercentage']

for i in percentages:
        df_joined[i] = round (df_joined[i] * 100, 2)

In [21]:
pings = df_joined.loc[:, 'allInPings':'onMyWayPings']
totalPings = pings.sum(axis=1)

df_joined.insert(15, 'totalPings', totalPings)

df_joined = df_joined.drop(pings, axis=1)

In [22]:
df_joined['gameLength'] = round (df_joined['gameLength'] / 60, 2)

df_joined['goldPerMinute'] = round (df_joined['goldPerMinute'], 2)

df_joined['kda'] = round (df_joined['kda'], 2)


In [23]:
df_joined

,kills,deaths,assists,kda,championName,individualPosition,killParticipation,summoner1Casts,summoner2Casts,largestMultiKill,totalDamageDealtToChampions,teamDamagePercentage,totalDamageTaken,damageTakenOnTeamPercentage,largestKillingSpree,totalPings,gameLength,goldEarned,goldPerMinute,wardsPlaced,wardsKilled,win
1,23,5,16,7.80,Tristana,Invalid,75.00,2,3,4,23585,22.87,12940,15.59,10,0,15.23,14571,957.01,0,0,True
2,6,10,28,3.40,Shen,Invalid,77.27,5,33,1,25707,15.52,58100,32.18,2,0,23.67,14124,596.83,0,0,False
3,15,13,24,3.00,Xayah,Invalid,60.94,5,6,2,27617,22.45,30754,16.14,3,1,19.41,15387,792.89,0,0,False
4,19,10,9,2.80,Jax,Invalid,54.90,6,36,2,26658,24.52,35250,22.04,3,7,23.13,17809,769.82,0,0,True
5,6,10,14,2.00,Camille,Invalid,50.00,4,16,2,13649,15.61,24784,12.59,2,0,16.54,12063,729.31,0,0,True
6,8,6,28,6.00,Corki,Invalid,72.00,4,5,2,36196,28.00,22136,19.16,4,1,21.00,14624,696.34,0,0,True
7,12,8,10,2.75,Caitlyn,Invalid,48.89,3,3,2,26665,21.40,18786,9.88,4,1,24.19,17602,727.58,0,0,False
8,14,8,16,3.75,Camille,Invalid,50.85,4,18,4,26262,20.47,27459,16.62,4,0,20.73,15641,754.38,0,0,True
9,17,2,17,17.00,Jhin,Invalid,75.56,3,2,4,18544,24.59,6627,13.98,17,2,13.90,12612,907.11,0,0,True
10,6,10,45,5.10,Rakan,Invalid,78.46,3,23,2,14063,7.96,49880,23.78,4,5,25.42,15715,618.35,0,0,False


In [25]:
df_joined.to_csv('game_history.csv', mode = 'x', sep=',')